# Projeto Ri - Kmeans

In [7]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as p
import random
import nltk as n
import matplotlib.pyplot as plt
import re
import seaborn as sb
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from scipy.spatial.distance import cdist
from collections import Counter
from sklearn.cluster import KMeans
from wordcloud import WordCloud
from sklearn.manifold import TSNE
from unicodedata import normalize

Nesse trecho de codigo fazemos toda uma pré processamento remove stopwords das noticias que poderam afetar a execução do Kmeans

In [9]:
def build(data):
    data["noticia"] = data.titulo + " " + data.subTitulo + " " + data.conteudo
    docs = []
    stop_words = set(stopwords.words('portuguese'))
    
    for index, doc in data.iterrows():
        doc.noticia = doc.noticia.lower()
        word_tokens = word_tokenize(doc.noticia)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        news = ' '.join(filtered_sentence)
        docs.append(clear_text(news))

    return docs

In [10]:
def clear_text(text):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

Como o conjunto de dados é muito grande temos que utilizar apenas 10 % do seu tamanho,e para isso utilizei a seleçao de documentos do nosso data-set de forma aleatoria.

In [26]:
file_csv = "https://raw.githubusercontent.com/wendleypf/recuperacao_informacao/master/projeto-ri/data-set/estadao_noticias_eleicao.csv"
data = p.read_csv(file_csv, encoding = "utf-8")
data = data.replace(np.NAN, "")

num_lines = len(data)
# ~10%
size = int(num_lines / 10)
skip_idx = random.sample(range(1, num_lines), num_lines - size)
data = p.read_csv(file_csv, skiprows=skip_idx, encoding = "utf-8")
data = data.replace(np.NAN, "")

## Processamento dos documentos

In [31]:
documents = build(data)

## Calculando TF-IDF de cada palavra nos documento

In [32]:
vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(documents)

## Qual o numero ideal de clusters ?

Podemos utilizar método Elbow, trata-se de uma técnica interessante para encontrar o valor ideal o número de clusters do nosso agrupamento utilizando o Kmeans.
Basicamente o que o método faz é testar a variância dos dados em relação ao número de clusters.
É considerado um valor ideal de k quando o aumento no número de clusters não representa um valor significativo de ganho.

In [34]:
K = range(1,10) # Definindo que o numero de clusters pode variar entre 1 e 10

In [ ]:
distortions = []
for k in K:
    kmeanModel = KMeans(n_clusters=k, init='k-means++', max_iter=10000, n_init=100).fit(vector.toarray())
    kmeanModel.fit(vector.toarray())
    distortions.append(sum(np.min(cdist(vector.toarray(), kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / vector.toarray().shape[0])

Graficamente podemos ver

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('Método do cotovelo para K ótimo')
plt.show()

## Execução do algoritmo Kmeans sobre os documentos

Depois da execuçao do método de Elbow podermos ver que o valor K seria.

In [ ]:
km = KMeans(n_clusters= 4, init='k-means++', max_iter=10000, n_init=100)
km.fit(vector.toarray())

Obtendo os centroids de cada grupo bem como os labels

In [ ]:
labels = km.labels_
centroids = np.array(km.cluster_centers_)

## WordCloud de cada grupo criado

In [ ]:
groups_dic = {}
groups_term_freq = {}

for i in range(len(documents)):
    groups_dic.setdefault(labels[i], []).append(documents[i])

for group in groups_dic.keys():
    groups_dic[group] = ' '.join(groups_dic[group])
    words = [word.lower() for word in n.word_tokenize(groups_dic[group])]
    freq_words = Counter(words)
    groups_term_freq[group] = freq_words.most_common()

    wordcloud = WordCloud(background_color='white', max_font_size=40).generate(groups_dic[group])
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title('Word Cloud do grupo ' + str(group))
    plt.axis("off")

## Top 4 palavras de cada grupo criado

In [ ]:
for group in groups_term_freq.keys():
    s = groups_term_freq[group][:4]
    y = []
    x = []
    for i in range(len(s)):
        x.append(s[i][0])
        y.append(s[i][1])

    plt.figure()
    plt.barh(x, y, color="blue")
    plt.xlabel("Número de ocorrências")
    plt.title("Palavras com a maior frequência no grupo " + str(group))